# Selection Functions
## Depth maps and selection functions

The simplest selection function available is the field MOC which specifies the area for which there is Herschel data. Each pristine catalogue also has a MOC defining the area for which that data is available.

The next stage is to provide mean flux standard deviations which act as a proxy for the catalogue's 5$\sigma$ depth.

For XID+ we use a 2$\sigma$ cut as is used to determine CIGALE fluxes.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000)
This notebook was executed on: 
2019-12-11 11:02:13.888658


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
import numpy as np
from pymoc import MOC
import healpy as hp
#import pandas as pd #Astropy has group_by function so apandas isn't required.
import seaborn as sns

import warnings
#We ignore warnings - this is a little dangerous but a huge number of warnings are generated by empty cells later
warnings.filterwarnings('ignore')

from herschelhelp_internal.utils import inMoc, coords_to_hpidx, flux_to_mag
from herschelhelp_internal.masterlist import find_last_ml_suffix, nb_ccplots

from astropy.io.votable import parse_single_table
import yaml

In [3]:
meta = yaml.load(open("./meta_main.yml", 'r'))

In [4]:
FIELD = meta['field']
#FILTERS_DIR = "/Users/rs548/GitHub/herschelhelp_python/database_builder/filters/"
FILTERS_DIR = "/opt/herschelhelp_python/database_builder/filters/"

In [5]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
#SUFFIX = find_last_ml_suffix()
SUFFIX = meta['final'].split('/')[-1].split('_')[-1].strip('.fits')

master_catalogue_filename = meta['final'].split('/')[-1]
master_catalogue = Table.read("{}/{}".format(OUT_DIR, master_catalogue_filename))

print("Depth maps produced using: {}".format(master_catalogue_filename))

ORDER = 10
#TODO write code to decide on appropriate order

field_moc = MOC(filename="../../dmu2/dmu2_field_coverages/{}_MOC.fits".format(FIELD))

Depth maps produced using: ELAIS-N1_20171016.fits


In [6]:
# Remove sources whose signal to noise ratio is less than five as these will have been selected using forced 
# photometry and so the errors will not refelct the RMS of the map 
# For xid+ saource use s/n >2 as that is criteria we use for CIGALE
for n,col in enumerate(master_catalogue.colnames):
    if col.startswith("f_spire") or col.startswith("f_pacs")or col.startswith("f_mips"):
        err_col = "ferr{}".format(col[1:])
        errs = master_catalogue[err_col]
        fluxes = master_catalogue[col]
        mask = fluxes/errs < 2.0
        master_catalogue[col][mask] = np.nan
        master_catalogue[err_col][mask] = np.nan
    elif col.startswith("f_"):
        err_col = "ferr{}".format(col[1:])
        errs = master_catalogue[err_col]
        fluxes = master_catalogue[col]
        mask = fluxes/errs < 5.0
        master_catalogue[col][mask] = np.nan
        master_catalogue[err_col][mask] = np.nan

## I - Group masterlist objects by healpix cell and calculate depths
We add a column to the masterlist catalogue for the target order healpix cell <i>per object</i>.

In [7]:
#Add a column to the catalogue with the order=ORDER hp_idx
master_catalogue.add_column(Column(data=coords_to_hpidx(master_catalogue['ra'],
                                                       master_catalogue['dec'],
                                                       ORDER), 
                                   name="hp_idx_O_{}".format(str(ORDER))
                                  )
                           )

In [8]:
# Convert catalogue to pandas and group by the order=ORDER pixel

group = master_catalogue.group_by(["hp_idx_O_{}".format(str(ORDER))])

In [9]:
#Downgrade the groups from order=ORDER to order=13 and then fill out the appropriate cells
#hp.pixelfunc.ud_grade([2599293, 2599294], nside_out=hp.order2nside(13))

## II Create a table of all Order=13 healpix cells in the field and populate it
We create a table with every order=13 healpix cell in the field MOC. We then calculate the healpix cell at lower order that the order=13 cell is in. We then fill in the depth at every order=13 cell as calculated for the lower order cell that that the order=13 cell is inside.

In [10]:
depths = Table()
depths['hp_idx_O_13'] = list(field_moc.flattened(13))

In [11]:
depths[:10].show_in_notebook()

idx,hp_idx_O_13
0,188743680
1,188743681
2,188743682
3,188743683
4,188743684
5,188743685
6,188743686
7,188743687
8,188743688
9,188743689


In [12]:
depths.add_column(Column(data=hp.pixelfunc.ang2pix(2**ORDER,
                     hp.pixelfunc.pix2ang(2**13, depths['hp_idx_O_13'], nest=True)[0],
                     hp.pixelfunc.pix2ang(2**13, depths['hp_idx_O_13'], nest=True)[1],
                     nest = True),
                     name="hp_idx_O_{}".format(str(ORDER))
                        )
                 )

In [13]:
depths[:10].show_in_notebook()

idx,hp_idx_O_13,hp_idx_O_10
0,188743680,2949120
1,188743681,2949120
2,188743682,2949120
3,188743683,2949120
4,188743684,2949120
5,188743685,2949120
6,188743686,2949120
7,188743687,2949120
8,188743688,2949120
9,188743689,2949120


In [14]:
for col in master_catalogue.colnames:
    if col.startswith("f_"):
        errcol = "ferr{}".format(col[1:])
        depths = join(depths, 
                      group["hp_idx_O_{}".format(str(ORDER)), errcol].groups.aggregate(np.nanmean),
                     join_type='left')
        depths[errcol].name = errcol + "_mean"
        depths = join(depths, 
                      group["hp_idx_O_{}".format(str(ORDER)), col].groups.aggregate(lambda x: np.nanpercentile(x, 90.)),
                     join_type='left')
        depths[col].name = col + "_p90"

depths[:10].show_in_notebook()

idx,hp_idx_O_13,hp_idx_O_10,ferr_wfc_u_mean,f_wfc_u_p90,ferr_ap_wfc_u_mean,f_ap_wfc_u_p90,ferr_megacam_u_mean,f_megacam_u_p90,ferr_ap_megacam_u_mean,f_ap_megacam_u_p90,ferr_suprime_g_mean,f_suprime_g_p90,ferr_ap_suprime_g_mean,f_ap_suprime_g_p90,ferr_megacam_g_mean,f_megacam_g_p90,ferr_ap_megacam_g_mean,f_ap_megacam_g_p90,ferr_gpc1_g_mean,f_gpc1_g_p90,ferr_ap_gpc1_g_mean,f_ap_gpc1_g_p90,ferr_wfc_g_mean,f_wfc_g_p90,ferr_ap_wfc_g_mean,f_ap_wfc_g_p90,ferr_suprime_r_mean,f_suprime_r_p90,ferr_ap_suprime_r_mean,f_ap_suprime_r_p90,ferr_gpc1_r_mean,f_gpc1_r_p90,ferr_ap_gpc1_r_mean,f_ap_gpc1_r_p90,ferr_wfc_r_mean,f_wfc_r_p90,ferr_ap_wfc_r_mean,f_ap_wfc_r_p90,ferr_megacam_r_mean,f_megacam_r_p90,ferr_ap_megacam_r_mean,f_ap_megacam_r_p90,ferr_gpc1_i_mean,f_gpc1_i_p90,ferr_ap_gpc1_i_mean,f_ap_gpc1_i_p90,ferr_suprime_i_mean,f_suprime_i_p90,ferr_ap_suprime_i_mean,f_ap_suprime_i_p90,ferr_wfc_i_mean,f_wfc_i_p90,ferr_ap_wfc_i_mean,f_ap_wfc_i_p90,ferr_gpc1_z_mean,f_gpc1_z_p90,ferr_ap_gpc1_z_mean,f_ap_gpc1_z_p90,ferr_wfc_z_mean,f_wfc_z_p90,ferr_ap_wfc_z_mean,f_ap_wfc_z_p90,ferr_megacam_z_mean,f_megacam_z_p90,ferr_ap_megacam_z_mean,f_ap_megacam_z_p90,ferr_suprime_z_mean,f_suprime_z_p90,ferr_ap_suprime_z_mean,f_ap_suprime_z_p90,ferr_suprime_n921_mean,f_suprime_n921_p90,ferr_ap_suprime_n921_mean,f_ap_suprime_n921_p90,ferr_gpc1_y_mean,f_gpc1_y_p90,ferr_ap_gpc1_y_mean,f_ap_gpc1_y_p90,ferr_suprime_y_mean,f_suprime_y_p90,ferr_ap_suprime_y_mean,f_ap_suprime_y_p90,ferr_ukidss_j_mean,f_ukidss_j_p90,ferr_ap_ukidss_j_mean,f_ap_ukidss_j_p90,ferr_ukidss_k_mean,f_ukidss_k_p90,ferr_ap_ukidss_k_mean,f_ap_ukidss_k_p90,ferr_irac_i1_mean,f_irac_i1_p90,ferr_ap_irac_i1_mean,f_ap_irac_i1_p90,ferr_irac_i2_mean,f_irac_i2_p90,ferr_ap_irac_i2_mean,f_ap_irac_i2_p90,ferr_irac_i3_mean,f_irac_i3_p90,ferr_ap_irac_i3_mean,f_ap_irac_i3_p90,ferr_irac_i4_mean,f_irac_i4_p90,ferr_ap_irac_i4_mean,f_ap_irac_i4_p90,ferr_mips_24_mean,f_mips_24_p90,ferr_pacs_green_mean,f_pacs_green_p90,ferr_pacs_red_mean,f_pacs_red_p90,ferr_spire_250_mean,f_spire_250_p90,ferr_spire_350_mean,f_spire_350_p90,ferr_spire_500_mean,f_spire_500_p90
,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy
0,163569021,2555765,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
1,163568991,2555765,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
2,163569013,2555765,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
3,163569015,2555765,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,

## Add deepest for main bands

For the standard common bands we find the minimum value across all specific bands.

In [15]:
standard = ['u', 'g', 'r', 'i', 'z', 'y', 'j', 'h', 'k', 'ks']

In [16]:
tot_bands = [column[2:] for column in master_catalogue.colnames 
             if (column.startswith('f_') & ~column.startswith('f_ap_'))]
ap_bands = [column[5:] for column in master_catalogue.colnames 
            if column.startswith('f_ap_') ]
bands = set(tot_bands) | set(ap_bands)
bands

{'gpc1_g',
 'gpc1_i',
 'gpc1_r',
 'gpc1_y',
 'gpc1_z',
 'irac_i1',
 'irac_i2',
 'irac_i3',
 'irac_i4',
 'megacam_g',
 'megacam_r',
 'megacam_u',
 'megacam_z',
 'mips_24',
 'pacs_green',
 'pacs_red',
 'spire_250',
 'spire_350',
 'spire_500',
 'suprime_g',
 'suprime_i',
 'suprime_n921',
 'suprime_r',
 'suprime_y',
 'suprime_z',
 'ukidss_j',
 'ukidss_k',
 'wfc_g',
 'wfc_i',
 'wfc_r',
 'wfc_u',
 'wfc_z'}

In [17]:
for col in depths.colnames:
    if depths[col].dtype == 'float64' or depths[col].dtype == 'float32':
        depths[col].fill_value = np.nan
        
depths = depths.filled()

In [18]:
for band in standard:
    print(band)
    #ap_bands
    list_of_ap_cols = np.array([ b if b.endswith('_{}'.format(band)) else None for b in ap_bands])
    list_of_ap_cols = list_of_ap_cols[list_of_ap_cols != None]
    list_of_ap_cols = ['ferr_{}_mean'.format(b) for b in list_of_ap_cols]
    print(list_of_ap_cols)
    if len(list_of_ap_cols) >0:
        depths['ferr_ap_{}_mean'.format(band)]  = np.nanmin([depths[t] for t in list_of_ap_cols], axis=0)
    #tot_bands
    list_of_tot_cols = np.array([ b if b.endswith('_{}'.format(band)) else None for b in tot_bands])
    list_of_tot_cols = list_of_tot_cols[list_of_tot_cols != None]
    list_of_tot_cols = ['ferr_{}_mean'.format(b) for b in list_of_tot_cols]
    print(list_of_tot_cols)
    if len(list_of_tot_cols) >0:
        depths['ferr_{}_mean'.format(band)]  = np.nanmin([depths[t] for t in list_of_tot_cols], axis=0)

u
['ferr_wfc_u_mean', 'ferr_megacam_u_mean']
['ferr_wfc_u_mean', 'ferr_megacam_u_mean']
g
['ferr_suprime_g_mean', 'ferr_megacam_g_mean', 'ferr_gpc1_g_mean', 'ferr_wfc_g_mean']
['ferr_suprime_g_mean', 'ferr_megacam_g_mean', 'ferr_gpc1_g_mean', 'ferr_wfc_g_mean']
r
['ferr_suprime_r_mean', 'ferr_gpc1_r_mean', 'ferr_wfc_r_mean', 'ferr_megacam_r_mean']
['ferr_suprime_r_mean', 'ferr_gpc1_r_mean', 'ferr_wfc_r_mean', 'ferr_megacam_r_mean']
i
['ferr_gpc1_i_mean', 'ferr_suprime_i_mean', 'ferr_wfc_i_mean']
['ferr_gpc1_i_mean', 'ferr_suprime_i_mean', 'ferr_wfc_i_mean']
z
['ferr_gpc1_z_mean', 'ferr_wfc_z_mean', 'ferr_megacam_z_mean', 'ferr_suprime_z_mean']
['ferr_gpc1_z_mean', 'ferr_wfc_z_mean', 'ferr_megacam_z_mean', 'ferr_suprime_z_mean']
y
['ferr_gpc1_y_mean', 'ferr_suprime_y_mean']
['ferr_gpc1_y_mean', 'ferr_suprime_y_mean']
j
['ferr_ukidss_j_mean']
['ferr_ukidss_j_mean']
h
[]
[]
k
['ferr_ukidss_k_mean']
['ferr_ukidss_k_mean']
ks
[]
[]


In [19]:
depths

hp_idx_O_13,hp_idx_O_10,ferr_wfc_u_mean,f_wfc_u_p90,ferr_ap_wfc_u_mean,f_ap_wfc_u_p90,ferr_megacam_u_mean,f_megacam_u_p90,ferr_ap_megacam_u_mean,f_ap_megacam_u_p90,ferr_suprime_g_mean,f_suprime_g_p90,ferr_ap_suprime_g_mean,f_ap_suprime_g_p90,ferr_megacam_g_mean,f_megacam_g_p90,ferr_ap_megacam_g_mean,f_ap_megacam_g_p90,ferr_gpc1_g_mean,f_gpc1_g_p90,ferr_ap_gpc1_g_mean,f_ap_gpc1_g_p90,ferr_wfc_g_mean,f_wfc_g_p90,ferr_ap_wfc_g_mean,f_ap_wfc_g_p90,ferr_suprime_r_mean,f_suprime_r_p90,ferr_ap_suprime_r_mean,f_ap_suprime_r_p90,ferr_gpc1_r_mean,f_gpc1_r_p90,ferr_ap_gpc1_r_mean,f_ap_gpc1_r_p90,ferr_wfc_r_mean,f_wfc_r_p90,ferr_ap_wfc_r_mean,f_ap_wfc_r_p90,ferr_megacam_r_mean,f_megacam_r_p90,ferr_ap_megacam_r_mean,f_ap_megacam_r_p90,ferr_gpc1_i_mean,f_gpc1_i_p90,ferr_ap_gpc1_i_mean,f_ap_gpc1_i_p90,ferr_suprime_i_mean,f_suprime_i_p90,ferr_ap_suprime_i_mean,f_ap_suprime_i_p90,ferr_wfc_i_mean,f_wfc_i_p90,ferr_ap_wfc_i_mean,f_ap_wfc_i_p90,ferr_gpc1_z_mean,f_gpc1_z_p90,ferr_ap_gpc1_z_mean,f_ap_gpc1_z_p90,ferr_wfc_z_mean,f_wfc_z_p90,ferr_ap_wfc_z_mean,f_ap_wfc_z_p90,ferr_megacam_z_mean,f_megacam_z_p90,ferr_ap_megacam_z_mean,f_ap_megacam_z_p90,ferr_suprime_z_mean,f_suprime_z_p90,ferr_ap_suprime_z_mean,f_ap_suprime_z_p90,ferr_suprime_n921_mean,f_suprime_n921_p90,ferr_ap_suprime_n921_mean,f_ap_suprime_n921_p90,ferr_gpc1_y_mean,f_gpc1_y_p90,ferr_ap_gpc1_y_mean,f_ap_gpc1_y_p90,ferr_suprime_y_mean,f_suprime_y_p90,ferr_ap_suprime_y_mean,f_ap_suprime_y_p90,ferr_ukidss_j_mean,f_ukidss_j_p90,ferr_ap_ukidss_j_mean,f_ap_ukidss_j_p90,ferr_ukidss_k_mean,f_ukidss_k_p90,ferr_ap_ukidss_k_mean,f_ap_ukidss_k_p90,ferr_irac_i1_mean,f_irac_i1_p90,ferr_ap_irac_i1_mean,f_ap_irac_i1_p90,ferr_irac_i2_mean,f_irac_i2_p90,ferr_ap_irac_i2_mean,f_ap_irac_i2_p90,ferr_irac_i3_mean,f_irac_i3_p90,ferr_ap_irac_i3_mean,f_ap_irac_i3_p90,ferr_irac_i4_mean,f_irac_i4_p90,ferr_ap_irac_i4_mean,f_ap_irac_i4_p90,ferr_mips_24_mean,f_mips_24_p90,ferr_pacs_green_mean,f_pacs_green_p90,ferr_pacs_red_mean,f_pacs_red_p90,ferr_spire_250_mean,f_spire_250_p90,ferr_spire_350_mean,f_spire_350_p90,ferr_spire_500_mean,f_spire_500_p90,ferr_ap_u_mean,ferr_u_mean,ferr_ap_g_mean,ferr_g_mean,ferr_ap_r_mean,ferr_r_mean,ferr_ap_i_mean,ferr_i_mean,ferr_ap_z_mean,ferr_z_mean,ferr_ap_y_mean,ferr_y_mean,ferr_ap_j_mean,ferr_j_mean,ferr_ap_k_mean,ferr_k_mean
,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,,,,,,,,,,,,,,,,
int64,int64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32
163569021,2555765,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

## III - Save the depth map table

In [21]:
depths.write("{}/depths_{}_{}.fits".format(OUT_DIR, FIELD.lower(), SUFFIX), overwrite=True)

## IV - Overview plots

### IV.a - Filters
First we simply plot all the filters available on this field to give an overview of coverage.

In [22]:
for b in bands:
    plt.plot(Table(data = parse_single_table(FILTERS_DIR + b + '.xml').array.data)['Wavelength']
            ,Table(data = parse_single_table(FILTERS_DIR + b + '.xml').array.data)['Transmission']
            , label=b)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Transmission')
plt.xscale('log')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Passbands on {}'.format(FIELD))

FileNotFoundError: [Errno 2] No such file or directory: '/opt/herschelhelp_python/database_builder/filters/pacs_green.xml'

### IV.a - Depth overview
Then we plot the mean depths available across the area a given band is available

In [ ]:
average_depths = []
for b in ap_bands:
    
    mean_err = np.nanmean(depths['ferr_ap_{}_mean'.format(b)])
    print("{}: mean flux error: {}, 3sigma in AB mag (Aperture): {}".format(b, mean_err, flux_to_mag(3.0*mean_err*1.e-6)[0]))
    average_depths += [('ap_' + b, flux_to_mag(1.0*mean_err*1.e-6)[0], 
                        flux_to_mag(3.0*mean_err*1.e-6)[0], 
                        flux_to_mag(5.0*mean_err*1.e-6)[0])]
    
for b in tot_bands:
    
    mean_err = np.nanmean(depths['ferr_{}_mean'.format(b)])
    print("{}: mean flux error: {}, 3sigma in AB mag (Total): {}".format(b, mean_err, flux_to_mag(3.0*mean_err*1.e-6)[0]))
    average_depths += [(b, flux_to_mag(1.0*mean_err*1.e-6)[0], 
                        flux_to_mag(3.0*mean_err*1.e-6)[0], 
                        flux_to_mag(5.0*mean_err*1.e-6)[0])]
    
average_depths = np.array(average_depths,  dtype=[('band', "<U16"), ('1s', float), ('3s', float), ('5s', float)])
    

In [ ]:
def FWHM(X,Y):
    
    half_max = max(Y) / 2.
    #find when function crosses line half_max (when sign of diff flips)
    #take the 'derivative' of signum(half_max - Y[])
    d = half_max - Y
    #plot(X,d) #if you are interested
    #find the left and right most indexes
    low_end = X[np.where(d < 0)[0][0]]
    high_end = X[np.where(d < 0)[0][-1]]
    return low_end, high_end, (high_end - low_end)


In [ ]:
data = []

for b in ap_bands:
    data += [['ap_' + b, Table(data = parse_single_table(FILTERS_DIR + b +'.xml').array.data)]]
    
for b in tot_bands:
    data += [[b, Table(data = parse_single_table(FILTERS_DIR + b +'.xml').array.data)]]

In [ ]:
wav_range = []
for dat in data:
    print(dat[0], FWHM(np.array(dat[1]['Wavelength']), np.array(dat[1]['Transmission'])))
    wav_range += [[dat[0], FWHM(np.array(dat[1]['Wavelength']), np.array(dat[1]['Transmission']))]]


In [ ]:
for dat in data:
    wav_deets = FWHM(np.array(dat[1]['Wavelength']), np.array(dat[1]['Transmission']))
    depth = average_depths['5s'][average_depths['band'] == dat[0]]
    #print(depth)
    plt.plot([wav_deets[0],wav_deets[1]], [depth,depth], label=dat[0])
    
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Depth')
plt.xscale('log')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Depths on {}'.format(FIELD))

### IV.c - Depth vs coverage comparison

How best to do this? Colour/intensity plot over area? Percentage coverage vs mean depth?

In [ ]:
for dat in data:
    wav_deets = FWHM(np.array(dat[1]['Wavelength']), np.array(dat[1]['Transmission']))
    depth = average_depths['5s'][average_depths['band'] == dat[0]]
    #print(depth)
    coverage = np.sum(~np.isnan(depths['ferr_{}_mean'.format(dat[0])]))/len(depths)
    plt.plot(coverage, depth, 'x', label=dat[0])
    
plt.xlabel('Coverage')
plt.ylabel('Depth')
#plt.xscale('log')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Depths (5 $\sigma$) vs coverage on {}'.format(FIELD))